<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 1 (демо).<br> Анализ данных по доходу населения UCI Adult</center>

**В задании предлагается с помощью Pandas ответить на несколько вопросов по данным репозитория UCI [Adult](https://archive.ics.uci.edu/ml/datasets/Adult) (качать данные не надо – они уже есть в репозитории).**

Уникальные значения признаков (больше информации по ссылке выше):
- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.   
- salary: >50K,<=50K

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('adult.data.csv')
data.head()

**1. Сколько мужчин и женщин (признак *sex*) представлено в этом наборе данных?**

In [ ]:
data.groupby(['sex']).size()

**2. Каков средний возраст (признак *age*) женщин?**

In [ ]:
female_age_avg = data.groupby(['sex']).get_group('Male')['age'].mean().astype('int32')
print(f"Average female age is: {female_age_avg} year(s)")

**3. Какова доля граждан Германии (признак *native-country*)?**

In [ ]:
german_person_count = data.groupby(['native-country']).get_group('Germany').shape[0]
german_person_ratio = german_person_count / data.shape[0]
print(f"Ratio of persons from Germany is: {german_person_ratio}")

**4-5. Каковы средние значения и среднеквадратичные отклонения возраста тех, кто получает более 50K в год (признак *salary*) и тех, кто получает менее 50K в год? **

In [ ]:
# Get groups by salary
reach_person_group = data.groupby(['salary']).get_group('>50K')
other_person_group = data.groupby(['salary']).get_group('<=50K')

# Get mean ages
reach_person_age_mean = reach_person_group['age'].mean()
other_person_age_mean = other_person_group['age'].mean()

# Get std
reach_person_age_std= reach_person_group['age'].std()
other_person_age_std = other_person_group['age'].std()

age_format = '.0f'
print(f"Reach persons have mean age {reach_person_age_mean:{age_format}} with std {reach_person_age_std:{age_format}}")
print(f"Other persons have mean age {other_person_age_mean:{age_format}} with std {other_person_age_std:{age_format}}")

**6. Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? (признак *education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters* или *Doctorate*)**

In [ ]:
# Set check list of education level
higher_education_list = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

# Check condition
reach_means_smart = reach_person_group['education'].isin(higher_education_list).all()

print(f"Answer on this question is: {reach_means_smart}")

**7. Выведите статистику возраста для каждой расы (признак *race*) и каждого пола. Используйте *groupby* и *describe*. Найдите таким образом максимальный возраст мужчин расы *Amer-Indian-Eskimo*.**

In [ ]:
# Get group stat
race_age_stat = data.groupby(['race', 'sex'])['age'].describe()

# Get Amer-Indian-Eskimo male max age
max_age = race_age_stat['max']['Amer-Indian-Eskimo']['Male']

print(f"Statistics:\n{race_age_stat}")
print(f"\nMax age of Amer-Indian-Eskimo is: {max_age}")

**8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак *marital-status*)? Женатыми считаем тех, у кого *marital-status* начинается с *Married* (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.**

In [ ]:
# Set list of statuses which are equal to married
married_statuses = ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse']

# Get mask for married persons
data['Married'] = data['marital-status'].isin(married_statuses)

# Get ratios
married_ratio = data[data['Married']]['salary'].value_counts(normalize=True).get('>50K', 0)
single_ratio = data[~data['Married']]['salary'].value_counts(normalize=True).get('>50K', 0)

if married_ratio > single_ratio:
    print("Reach person ratio is bigger for married persons.")
elif married_ratio < single_ratio:
    print("Reach person ratio is bigger for single persons.")
else:
    print("Ratios are identical.")

**9. Какое максимальное число часов человек работает в неделю (признак *hours-per-week*)? Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?**

In [ ]:
max_hours_per_week = data['hours-per-week'].max()
person_count = (data['hours-per-week'] == max_hours_per_week).sum()

data['hard_worker'] = (data['hours-per-week'] == max_hours_per_week)

reach_percentage = data[data['hard_worker']]['salary'].value_counts(normalize=True).get('>50K', 0) * 100

print(f"Max hours per week is {max_hours_per_week}. {person_count} persons do it and {reach_percentage:.1f}% of them are reach persons.")

**10. Посчитайте среднее время работы (*hours-per-week*) зарабатывающих мало и много (*salary*) для каждой страны (*native-country*).**

In [ ]:
average_hours_per_week = data.groupby(['native-country', 'salary'])['hours-per-week'].mean()
average_hours_per_week